# Advanced Optimization - Large Scale Optimization

Matthew Brun

In [ ]:
# using Pkg;
# Pkg.add.(["Convex", "JuMP", "Images", "DelimitedFiles"]);

---
$\textsf{Session 2: Large-Scale Optimization}$
---

* How to identify structure in large-scale problems
  
* Algorithms for parallelizing structured optimization problems
  
* Parallel computing in `Julia`

* Optimization under memory contraints

## Problem Structure and Decomposition

* **Key Idea**: Identify substructures of problem that are easy to solve and repeatedly solve structured subproblems

* What is easy to solve (relative to full problem)?

    - Contains fewer variables/constraints than full problem

    - Has efficient algorithm (dynamic programming, network flow)

    - Tractable class of problem (e.g. MINLP -> MILP + NLP)

    - Limited interaction with other parts of the problem

* How do we "combine" solutions from multiple subproblems?

    - Utilize special algorithms designed for decomposition

    - May need to solve subproblem instances many times or accept degraded solution quality

## Structure in Linear Programs

* Linear programs have objectives that are separable in each variable (objective value is the sum of the objective contributed by each varaible)
  
  - This differs from functions with variable interactions, like $(x_1 + x_2)^2$

* Constraints are given by $A x \leq b$

* Interactions between variables are shown in the sparsity pattern of matrix $A$

    - Sparsity pattern = elements with nonzero entries\
    <img src="figures/largescale/sparsity_pattern.png" alt="image-20220117142714899" style="width: 300px;" />\
    (Wikipedia)
  
* Sparsity pattern generalizes to nonlinear programs, consider indicating which variables appear in a constraint

* Arbitrary ordering of matrix rows/columns may not reveal sparsity pattern 


## Common Sparsity Patterns

Group variables and constraints into "blocks"

Common patterns:

1. **Loose Coupling**: A few constraints link all variables, remaining constraints apply to blocks of variables\
    <img src="figures/largescale/loose_sparsity.png" alt="image-20220117142714899" style="width: 500px;" />

    - e.g. multiple locations with resource constraints
    - Compatible with column generation algorithms

1. **Two-Stage**: Some variables appear in many constraints, other variables appear in blocks of constraints\
    <img src="figures/largescale/twostage_sparsity.png" alt="image-20220117142714899" style="width: 500px;" />

    - e.g. two-stage stochastic program with recourse
    - Compatible with row generation/Benders' decomposition
   
1. **Overlapping**: Constraints apply to a block of variables and "adjacent" blocks\
    <img src="figures/largescale/overlapping_sparsity.png" alt="image-20220117142714899" style="width: 500px;" />

    - e.g. multistage programming
    - Compatible with stochastic dual dynamic programming (SDDP)

Note: it is possible to convert between different patterns.  One approach is to introduce "copies" of variabes and identity constraints ($x = y$)

## Stochastic Programming Example: Farmer's Problem [(Birge & Louveaux)](https://link.springer.com/book/10.1007/978-1-4614-0237-4)

Consider a farmer who needs to decide what crops to plant, with the following considerations:

* The crops are wheat, corn, and sugar cane
  
* 500 acres of land are available

* Planting costs per acre are $\$150$, $\$230$, and $\$260$, respectively

* Wheat and corn can be sold for $\$170$ and $\$150$/ton, respectively

* $6000$ tons of sugar cane can be sold for $\$36$/ton, additional sugar cane can only be sold for $10/ton

* The farmer needs $200$ tons of wheat and $240$ tons of corn to feed cattle

* Additional wheat and corn can be purchased for $40\%$ more than the sale price

* Crop yield per acre planted depends on the weather conditions ("good", "neutral", or "bad") and is not known in advance; each condition is equally likely.  Yields are summarized in the following table:

| Weather | Wheat Yield  | Corn Yield   | Sugar Cane Yield |
| ------- | ------------ | ------------ | ---------------- |
| Good    | 3 ton/acre   | 3.6 ton/acre | 24 ton/acre      |
| Neutral | 2.5 ton/acre | 3 ton/acre   | 20 ton/acre      |
| Bad     | 2 ton/acre   | 2.4 ton/acre | 16 ton/acre      |

Let variables $x_w$, $x_c$, and $x_s$ denote the acres planted of each crop.  Variables $y_w$, $y_c$, and $y_s$ give the tons of each crop sold, and $z_w$ and $z_c$ the tons of wheat and corn purchased for feed.  Finally, $w$ gives the amount of excess sugar cane sold at the lower price.  As the actions $y,z,w$ depend on the crop yield, these variables are also indexed by scenario $s \in \{g,n,b\} := S$.

The problem can be modeled in the extensive form as follows:

$$
\begin{array}{ll} \text{minimize} & 150 x_w + 230 x_c + 260 x_s + \frac{1}{3}\left (\sum_{s \in S} 170 (1.4 z_w^s - y_w^s) + 150 (1.4 z_c^s - y_c^s) - 36 y_s^s - 10 w_s^s \right )\\
\text{subject to} & x_w + x_c + x_s \leq 500 \\
                    & x,y,z,w \geq 0\\
                    & \textbf{Good Scenario}\\
                    & 3 x_w - y_w^g + z_w^g \geq 200\\
                    & 3.6 x_c - y_c^g + z_c^g \geq 240\\
                    & 24 x_s - y_s^g - w_s^g \geq 0\\
                    & y_s^g \leq 6000\\
                    & \textbf{Neutral Scenario}\\
                    & 2.5 x_w - y_w^n + z_w^n \geq 200\\
                    & 3 x_c - y_c^n + z_c^n \geq 240\\
                    & 20 x_s - y_s^n - w_s^n \geq 0\\
                    & y_s^n \leq 6000\\
                    & \textbf{Bad Scenario}\\
                    & 2 x_w - y_w^b + z_w^b \geq 200\\
                    & 2.4 x_c - y_c^b + z_c^b \geq 240\\
                    & 16 x_s - y_s^b - w_s^b \geq 0\\
                    & y_s^b \leq 6000\\
 \end{array}
$$


In [ ]:
include("data//largescale//farmer_data.jl");

using JuMP, Gurobi
model = Model(Gurobi.Optimizer)

@variable(model, x[crops] >= 0)
@variable(model, y[crops, scenarios] >= 0)
@variable(model, z[crops, scenarios] >= 0)
@variable(model, w[crops, scenarios] >= 0)

fix.([z["sugar",s] for s in scenarios], 0, force=true)

@constraint(model, sum(x) <= land_size)
@constraint(model, [s in scenarios], y["sugar", s] <= sugar_threshold)
@constraint(model, [i in crops, s in scenarios], yield[(i,s)] * x[i] + z[i,s] - y[i,s] - w[i,s] >= feed_requirement[i])

@objective(model, Min, 1/3 * sum(
        sum(planting_cost[i] * x[i] + sale_price[i] * (purchase_multiplier * z[i,s] - y[i,s]) for i in crops) - sugar_second_price * w["sugar",s]
    for s in scenarios )
)

optimize!(model)

The optimal solution is as follows:

In [ ]:
for c in crops
    println(c, ": ", value(x[c]))
end

## Algorithms for Decomposition

### Benders' Decomposition (BD)

* Tailored for problems with two-stage structure

* Replace each "block" with its value function

* Iteratively approximate the value function with linear cuts
  - Valid cuts are built from dual multipliers of each block

* Consider the initial problem in the "extensive form":
\begin{array}{ll}
\text{minimize} & f(x) + \sum_{\omega \in \Omega} g_\omega (y_\omega)\\
\text{subject to} & (x,y_\omega) \in \mathcal{Y}_\omega \quad \forall  \omega \in \Omega.
\end{array}

* We instead solve the "master problem":
\begin{array}{ll}
\text{minimize} & f(x) + \sum_{\omega \in \Omega} v_\omega\\
\text{subject to} & v_{\omega} \geq \pi_j^T x \quad \forall j \in J_\omega,\ \omega \in \Omega.
\end{array}

* For an incumbent solution $x'$, a new cut $\pi$ can be computed from the solution to the problem
\begin{array}{ll}
\text{minimize} & g_{\omega}(y)\\
\text{subject to} & (x',y) \in \mathcal{Y}_\omega.
\end{array}

* The master problem has fewer variables than the extensive form

* The subproblems are separable and can be solved in parallel for each $\omega$

### Alternating Direction Method of Multipliers (ADMM)

* Tailored for problems with linking constraints

    - e.g. $x_1 = x_2$
  
* Add linear and quadratic penalty terms

* Iteratively solve subproblems then update penalty multipliers

* Consider the extensive form, with two blocks (for convenience):
\begin{array}{ll}
\text{minimize} & f(x) + g(y)\\
\text{subject to} & A x + B y = 0.
\end{array}

* We instead consider the augmented Lagrangian function $$L(x,y,\lambda) = f(x) + g(y) + \lambda^T (A x + B y) + \frac{\rho}{2} \|A x + B y \|^2$$

* ADMM takes the following iteration:
    1.  $x_{k+1} \in \underset{x}{\text{argmin}}\ L(x,y_k,\lambda_k)$
    2.  $y_{k+1} \in \underset{x}{\text{argmin}}\ L(x_{k+1},y_k,\lambda_k)$
    3.  $\lambda_{k+1} = \lambda_k + \rho (A x_{k+1} + B y_{k+1})$

* These are **Gauss-Seidel** updates (not **Jacobian**), and are not parallelizable

* If $f$ or $g$ are decomposable, the optimization within each step may be parallelized

* Subproblems have quadratic objective terms

### Progressive Hedging (PH)

* Tailored for multistage stochastic problems with **nonanticipativity constraints**

* Similar scheme to ADMM, penalizes difference from average iterate

* Consider the extensive form:
\begin{array}{ll}
\text{minimize} & \sum_{\omega \in \Omega} p_\omega (f(x_\omega) + g_\omega(y_\omega))\\
\text{subject to} &x_{\omega_1} = x_{\omega_2} \quad \forall (\omega_1,\omega_2) \in \Omega^2.
\end{array}

    - Require that $\sum_{\omega \in \Omega} p_{\omega} = 1$
  
* PH takes the following iteration:
    1.  $(x_{\omega}^{k+1},y_{\omega}^{k+1}) \in \underset{x,y}{\text{argmin}}\ f(x) + g_{\omega} (y) + \langle \lambda_\omega^k, x \rangle + \frac{\rho}{2} \|x - \overline{x}^{k} \|^2$ for all $\omega$
    2.  $\overline{x}^{k+1} = \sum_{\omega \in \Omega} p_\omega x_{\omega}^{k+1}$
    3.  $\lambda_\omega^{k+1} = \lambda_\omega^k + \rho (x_\omega^{k+1} - \overline{x}^{k+1})$

* Subproblems are parallelizable over scenarios $\omega$

* Subproblems have quadratic objective terms

### Convergence 

* If the extensive form is convex, methods ADMM and PH converge to an optimal solution

* Under additional assumptions (e.g. linearity, strong duality) BD converges to an optimal solution

* If nonconvexity is present, convergence is not guaranteed

    - Methods may still work as heuristics

## Parallel Computing for Implementation

* Parallel computing runs multiple operations at the same time

* Leverages modern computing architectures (multi-core CPU, multi-CPU HPC)

* Adds runtime overhead from additional communication requirements

### Multithreading

* Julia natively supports multithreaded for loops

* Start Julia with the argument `-t/--threads`, specifying the number of threads
    
    - Can change default thread number in VSCode settings (search Julia threads)

* Prefacing a for loop with `Threads.@threads` runs a multithreaded for loop

* **Thread safety**: avoid changing data that will be accessed by other threads, as order is not guaranteed

    - Using custom tools (data locking, atomic operations), issues can be avoided
  
    - When calling other programs (e.g. solvers, Gurobi...) it is good practice to disable multithreading within the program

Check the number of available threads:

In [ ]:
Threads.nthreads()

Run multithreaded for loop that tracks thread number by iteration:

In [ ]:
n = 20
a = zeros(n)
Threads.@threads for i = 1:n
    a[i] = Threads.threadid()
end
a

Unsafe multithreaded loop may give incorrect result.  
The following code sums the values 1 to n in a threaded for loop (`s2`) and compares the result to the true value (`s1`).  Across repeated runs, `s2` may not give the correct result.

In [ ]:
n = 200
s1 = sum(1:n)
function sum_threads(n)
    s2 = 0
    Threads.@threads for i = 1:n
        s2 = s2 + i
    end
    return s2
end
s2 = sum_threads(n)
println("s1: ", s1)
println("s2: ", s2)

### Decomposed Stochastic Programming Example
 
* In this example, we revisit the Farmer's problem

* Problem is decomposed across scenarios by the progressive hedging algorithm

* Subproblems are solved in parallel with multithreading

* Note: Gurobi multithreading is disabled by setting the `Threads` solver argument

First, generate JuMP models for each scenario.  We construct a function that takes as input a scalar multiplier on the crop yields and returns the JuMP model for the scenario.  Then, we build a list containing the JuMP models.  Also, the objective functions are stored in a vector to facilitate the addition of penalty terms.

In [ ]:
include("data//largescale//farmer_data.jl");

using JuMP, Gurobi, Statistics

function build_scenario_problem(scenario_multiplier)

    model = Model(optimizer_with_attributes(
        Gurobi.Optimizer,
        "Threads"=>1
    ))

    @variable(model, x[crops] >= 0)
    @variable(model, y[crops] >= 0)
    @variable(model, z[crops] >= 0)
    @variable(model, w[crops] >= 0)

    fix.(z["sugar"], 0, force=true)

    @constraint(model, sum(x) <= land_size)
    @constraint(model, y["sugar"] <= sugar_threshold)
    @constraint(model, [i in crops], (1 + scenario_multiplier) * yield[(i,"neutral")] * x[i] + z[i] - y[i] - w[i] >= feed_requirement[i])

    @objective(model, Min, sum(planting_cost[i] * x[i] + sale_price[i] * (purchase_multiplier * z[i] - y[i]) for i in crops) - sugar_second_price * w["sugar"])
    return model

end

scenario_ids = 1:3
scenario_multipliers = -0.2:0.2:0.2
scenario_problems = [build_scenario_problem(scenario_multipliers[i]) for i in scenario_ids]
scenario_objectives = [objective_function(scenario_problems[i]) for i in scenario_ids]

Next, initialize the objects (penalty terms and average iterate) needed for PH

In [ ]:
xbar = Dict([(c,0.) for c in crops])
lambda = Dict([((i,c),0.) for c in crops, i in scenario_ids])
rho = 10

An iteration of PH proceeds as follows:

In [ ]:
for i in scenario_ids
    @objective(
        scenario_problems[i], 
        Min, 
        scenario_objectives[i] + sum(lambda[(i,c)] * scenario_problems[i][:x][c] + rho / 2 * (scenario_problems[i][:x][c] - xbar[c])^2 for c in crops)
    )

    optimize!(scenario_problems[i])

end

xbar = Dict([(c,mean(value.(scenario_problems[i][:x][c]) for i in scenario_ids)) for c in crops])

for c in crops, i in scenario_ids
    lambda[(i,c)] += rho * (value.(scenario_problems[i][:x][c]) - xbar[c])
end


The full code is as follows.  Note that a termination criterion is added, which checks convergence of the subproblem variables to the mean.  Additionally, the loop to solve the subproblems is multithreaded.

In [ ]:
include("data//largescale//farmer_data.jl");

using JuMP, Gurobi, Statistics

function build_scenario_problem(scenario_multiplier)

    model = Model(optimizer_with_attributes(
        Gurobi.Optimizer,
        "Threads"=>1,
        MOI.Silent()=>true
    ))

    @variable(model, x[crops] >= 0)
    @variable(model, y[crops] >= 0)
    @variable(model, z[crops] >= 0)
    @variable(model, w[crops] >= 0)

    fix.(z["sugar"], 0, force=true)

    @constraint(model, sum(x) <= land_size)
    @constraint(model, y["sugar"] <= sugar_threshold)
    @constraint(model, [i in crops], (1 + scenario_multiplier) * yield[(i,"neutral")] * x[i] + z[i] - y[i] - w[i] >= feed_requirement[i])

    @objective(model, Min, sum(planting_cost[i] * x[i] + sale_price[i] * (purchase_multiplier * z[i] - y[i]) for i in crops) - sugar_second_price * w["sugar"])
    return model

end

scenario_ids = 1:3
scenario_multipliers = -0.2:0.2:0.2
scenario_problems = [build_scenario_problem(scenario_multipliers[i]) for i in scenario_ids]
scenario_objectives = [objective_function(scenario_problems[i]) for i in scenario_ids]

xbar = Dict([(c,0.) for c in crops])
lambda = Dict([((i,c),0.) for c in crops, i in scenario_ids])
rho = 1

max_error = Inf

itx = 1
while max_error > 1e-8
    Threads.@threads for i in scenario_ids
        @objective(
            scenario_problems[i], 
            Min, 
            scenario_objectives[i] + sum(lambda[(i,c)] * scenario_problems[i][:x][c] + rho / 2 * (scenario_problems[i][:x][c] - xbar[c])^2 for c in crops)
        )

        optimize!(scenario_problems[i])

    end

    xbar = Dict([(c,mean(value.(scenario_problems[i][:x][c]) for i in scenario_ids)) for c in crops])

    max_error = 0
    for c in crops, i in scenario_ids
        term_error = (value.(scenario_problems[i][:x][c]) - xbar[c])
        lambda[(i,c)] += rho * term_error

        max_error = max(max_error, abs(term_error))
    end
    println("Iter ", itx, ": ", max_error)
    itx += 1

end


Observe that the solution aligns with that returned by the extensive form:

In [ ]:
xbar

### Message Passing Interface/`Distributed`

* Allows for parallel processing across multiple nodes

* Each task runs its own instance of the Julia program, behavior differs based on the process number

* More flexible than multithreading

    - A process doesn't need to wait for other threads to complete before proceeding
    - Data can be shared across processes (message passsing)

* Higher development cost than multithreading

    - Program design requires more thought and customization

* Message Passing Interface (MPI) is a parallel computing program that is accessible through many programming languges

    - Package `MPI.jl` provides wrappers for Julia
    - `mpi4py` is the corresponding Python package

* `Distributed.jl` gives a high-level Julia alternative to MPI
    - Number of processes is controlled with argument `-p`
    - e.g. `julia -p 4`

* MPI and Distributed are beyond the scope of this lesson, for more information, see:
    - [Julia documentation on distributed computing](https://docs.julialang.org/en/v1/manual/distributed-computing/)
    - [Github example for Distributed.jl](https://github.com/Arpeggeo/julia-distributed-computing)

## Optimization under Memory Constraints

* Some problems require large amounts of data that cannot be simultaneously loaded into memory (RAM)

* e.g. large statistical problems, big data

* Need to use algorithms and data structures that only use part of the data in memory at each step

### Stochastic Gradient Descent (SGD)

* Heuristic method used for large-scale nonconvex optimization in machine learning applications

    - Applied for deep learning/neural network training

* Samples **batches** of training data (stochastic), computes gradient on the batch (gradient descent)

* Popular implementations (`Keras`, `TensorFlow`, `PyTorch`...) have implementations for memory management when loading data

    - Typically, only a single batch is stored in memory

### Block Coordinate Descent (BCD)

* Method for convex optimization that applies **Gauss-Siedel** updates to subsets of coordinates (blocks)

* Consider the extensive form:
\begin{array}{ll}
\text{minimize} & f(x_1,\ldots,x_n)\\
\end{array}

* The method uses the following update for all $i$:\
    $x_i^{k+1} \in \underset{x_i}{\text{argmin}}\ f(x_1^{k+1},\ldots,x_{i-1}^{k+1},x_i,x_{i+1}^{k},\ldots,x_n^{k})$

* The update for the i-th coordinate fixes the next iterate for all preceeding coordinates and uses the current iterate for all following coordinates

* BCD converges to the optimal solution in the convex smooth setting

* Only the necessary data for solving for the i-th coordinate needs to be held in memory

    - For structured problems, this can significantly reduce memory requirements

### Large-Scale Linear Regression with BCD

* Linear regression solves the problem
\begin{array}{ll}
\underset{\theta}{\text{minimize}} & \|A \theta - b\|_2^2\\
\end{array}

    - $A$ is a matrix of observations of the independent variables and $b$ is the vector of dependent variable observations

    - Linear regression has a closed form solution: $\theta^* = (A^T A)^{-1} A^T b$

* If matrix $A$ contains too many variables/observations, it may be too large to be stored in memory

* Applying BCD intelligently allows us to use a single row of $A$ for each coordinate update

* Store the residual $r = A \theta - b$

* For incumbent residual $r$, the block update for coodinate i can be solved by\
$\delta^* \in \underset{\delta}{\text{argmin}} \|r + \delta A_i\|_2^2 \qquad \text{and} \qquad \theta = \theta + \delta^* \mathbb{e}_i \qquad \text{and} \qquad r = r + \delta^* A_i$\
where $A_i$ is the $i$-th column of $A$

* Note that $\delta^*$ has a closed-form solution as a linear regression with a single variable

### Wine Quality Dataset

* We apply this method to the [wine quality dataset](https://archive.ics.uci.edu/dataset/186/wine+quality)

    - Dependent variable: wine quality score on [0,10]
    - Independent variables: qualities including acidity, pH, alcohol content...

In [ ]:
## This code converts the full CSV file into a CSV file for each column

# using DelimitedFiles
# mat = readdlm("data//largescale//winequality-white.csv", ';')
# for i in 1:length(mat[1,:])
#     file_name = string("data//largescale//winequality-white//", replace(mat[1,i], " "=>"_"), ".csv")
#     writedlm(file_name, mat[2:end,i], ';')
# end

First, we identify the predictor columns and the observations of the dependent variable.  We also initialize the model parameters to $0$ and take $r = b$, so that the residuals correspond to the paramters.

In [ ]:
using DelimitedFiles, LinearAlgebra
column_names = setdiff(replace.(readdir("data//largescale//winequality-white//"), ".csv" => ""), ["quality"])
b = readdlm("data//largescale//winequality-white//quality.csv")

m = length(column_names)
n = length(b)

r = copy(b)
r_prev = copy(r)
θ = Dict((c_name,0.) for c_name in column_names)

A single iteration of block updates can be computed by the following code.  Each iteration loads a single column into memory, computes the change in the regression parameter, and updates the residual.

In [20]:
for c_name in column_names
    c_data = readdlm(string("data//largescale//winequality-white//", c_name, ".csv"))
    δ = - sum(c_data .* r) / sum(c_data .* c_data)
    θ[c_name] += δ
    r += δ .* c_data
end

The method iterates until no significant residual improvement is observed:

In [ ]:
norm(r_prev) - norm(r) < 1e-7

The full implementation is as follows

In [ ]:
using DelimitedFiles, LinearAlgebra
column_names = setdiff(replace.(readdir("data//largescale//winequality-white//"), ".csv" => ""), ["quality"])
b = readdlm("data//largescale//winequality-white//quality.csv")

m = length(column_names)
n = length(b)

r = -copy(b)
θ = Dict((c_name,0.) for c_name in column_names)

itx = 1
while true
    r_prev = r
    for c_name in column_names
        c_data = readdlm(string("data//largescale//winequality-white//", c_name, ".csv"))
        δ = sum(c_data .* r) / sum(c_data .* c_data)
        θ[c_name] += -δ
        r -= δ .* c_data
    end
    if norm(r_prev) - norm(r) < 1e-7
        break
    end
    itx += 1
end

println("# Iter: ", itx)
println("Optimal Solution:")
θ

As this problem is small enough to fit in memory, we can compute the optimal solution via the closed-form solution:

In [ ]:
data_in = readdlm("data//largescale//winequality-white.csv", ';')
column_names = replace.(data_in[1,:], " " => "_")
A = Matrix{Float64}(data_in[2:end,1:end-1])
b = data_in[2:end,end]
θ_true = inv(A' * A) * A' * b

We compare the difference in parameters as follows, and find that block coordinate descent identifies a comparable solution

In [ ]:
println("Absolute Coefficient Error:")
for i = 1:length(column_names)-1
    println("\t", column_names[i], ": ", abs.(θ_true[i] - θ[column_names[i]]))
end

In [ ]:
println("Relative Coefficient Error:")
for i = 1:length(column_names)-1
    println("\t", column_names[i], ": ", abs.(θ_true[i] - θ[column_names[i]]) / θ_true[i])
end